Directly load predictions of LGBM (latest) and pseudo label it for <0.2 and >0.8 ; followed by retraining the entire model again

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import time

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import lightgbm as lgb

import utils

import matplotlib.pyplot as plt

In [41]:
train_file = '../data/train'
test_file = '../data/test'
pred_ = '../results/submission_lgb.csv'

df_train = utils.read_pickles(train_file)
df_test = utils.read_pickles(test_file)
target = df_train.pop('target')


# df_train = pd.read_csv(train_file)
# df_test = pd.read_csv(test_file)

100%|██████████| 3/3 [00:00<00:00,  3.46it/s]


In [42]:
preds = pd.read_csv(pred_)
preds['ssl'] = -1

preds['ssl'][preds['target'] < 0.03] = 0
preds['ssl'][preds['target'] > 0.5] = 1

# preds.drop(preds[preds['ssl']==-1].index, inplace=True)

In [43]:
preds['ssl'].value_counts()

-1    104788
 0     89224
 1      5988
Name: ssl, dtype: int64

In [44]:
df_test['target'] = preds['ssl']

In [45]:
df_test.drop(df_test[df_test['target']==-1].index, inplace=True)

In [46]:
test_target = df_test.pop('target')

In [47]:
target.shape[0]+test_target.shape[0]

295212

In [48]:
target = pd.concat([target, test_target])

In [49]:
df = pd.concat([df_train, df_test])

In [50]:
df.drop(['ID_code'],axis=1, inplace=True)

In [51]:
df.columns

(295212, 339)

In [52]:
params = {'num_leaves': 9,
         'min_data_in_leaf': 42,
         'objective': 'binary',
         'max_depth': 16,
         'learning_rate': 0.0123,
         'boosting': 'gbdt',
         'bagging_freq': 5,
         'bagging_fraction': 0.8,
         'feature_fraction': 0.8201,
         'bagging_seed': 11,
         'reg_alpha': 1.728910519108444,
         'reg_lambda': 4.9847051755586085,
         'random_state': 42,
         'metric': 'auc',
         'verbosity': -1,
         'subsample': 0.81,
         'min_gain_to_split': 0.01077313523861969,
         'min_child_weight': 19.428902804238373,
         'num_threads': 4}

In [ ]:
df_test = utils.read_pickles(test_file)
test_id = df_test.pop('ID_code')

In [55]:
%%time
fold_n=3

folds = StratifiedKFold(n_splits=fold_n, shuffle=True, random_state=30)
y_pred_lgb = np.zeros(len(df_test))
oof = np.zeros(len(df))
feature_importance_df = pd.DataFrame()
features = list(df.columns)

for fold_, (train_index, valid_index) in enumerate(folds.split(df, target)):
    print('Fold', fold_, 'started at', time.ctime())
    X_train, X_valid = df.iloc[train_index], df.iloc[valid_index]
    y_train, y_valid = target.iloc[train_index], target.iloc[valid_index]
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid)
        
    lgb_model = lgb.train(params, train_data, num_boost_round=10000,
                    valid_sets = [train_data, valid_data], verbose_eval = 1000, early_stopping_rounds = 100)
    
    oof[valid_index] = lgb_model.predict(df.iloc[valid_index], num_iteration = lgb_model.best_iteration)
    
    y_pred_lgb += lgb_model.predict(df_test, num_iteration = lgb_model.best_iteration)/fold_n
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = lgb_model.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    

print("CV score: {:<8.5f}".format(roc_auc_score(target, oof)))

100%|██████████| 3/3 [00:00<00:00,  8.78it/s]


Fold 0 started at Sun Mar 24 21:55:02 2019
Training until validation scores don't improve for 100 rounds.
[1000]	training's auc: 0.931984	valid_1's auc: 0.919135
[2000]	training's auc: 0.949554	valid_1's auc: 0.933682
[3000]	training's auc: 0.957425	valid_1's auc: 0.939025
[4000]	training's auc: 0.961981	valid_1's auc: 0.941795
[5000]	training's auc: 0.965031	valid_1's auc: 0.943222
[6000]	training's auc: 0.96768	valid_1's auc: 0.943886
[7000]	training's auc: 0.970171	valid_1's auc: 0.944183
Early stopping, best iteration is:
[7560]	training's auc: 0.971536	valid_1's auc: 0.944272
Fold 1 started at Sun Mar 24 22:06:43 2019
Training until validation scores don't improve for 100 rounds.
[1000]	training's auc: 0.932815	valid_1's auc: 0.91869
[2000]	training's auc: 0.950212	valid_1's auc: 0.933315
[3000]	training's auc: 0.957911	valid_1's auc: 0.938831
[4000]	training's auc: 0.962301	valid_1's auc: 0.941365
[5000]	training's auc: 0.965365	valid_1's auc: 0.942626
[6000]	training's auc: 0.96

In [56]:
submission_lgb = pd.DataFrame({
        "ID_code": test_id,
        "target": y_pred_lgb
    })
submission_lgb.to_csv('../results/ssl_lgb.csv', index=False)